In [ ]:
# 必要なライブラリのインポート
import sys
import os
sys.path.append('../../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
%matplotlib inline

# 自作モジュールのインポート
from src.data_fetcher import DataFetcher
from src.technical_indicators import TechnicalIndicators
from src.sakata_patterns import SakataPatterns

import warnings
warnings.filterwarnings('ignore')


In [ ]:
# データの取得と準備
# サンプルデータの生成（実際のプロジェクトではAPIからデータを取得）
np.random.seed(42)
dates = pd.date_range(start='2022-01-01', periods=2000, freq='D')
data = pd.DataFrame({
    'Open': np.random.randn(2000).cumsum() + 100,
    'High': np.random.randn(2000).cumsum() + 102,
    'Low': np.random.randn(2000).cumsum() + 98,
    'Close': np.random.randn(2000).cumsum() + 100,
    'Volume': np.random.randn(2000).cumsum() + 1000000
}, index=dates)

# 価格の整合性を確保
for i in range(len(data)):
    data.iloc[i, 1] = max(data.iloc[i, [0, 1, 2, 3]])  # High
    data.iloc[i, 2] = min(data.iloc[i, [0, 1, 2, 3]])  # Low

# テクニカル指標を追加
df = TechnicalIndicators.add_bollinger_bands(data)

print(f"分析データ準備完了: {len(df)}件")
df.head()


In [ ]:
# 酒田五法パターンの検出
sakata = SakataPatterns(df)

# パターンの要約を取得
summary = sakata.get_pattern_summary()

print("=== 酒田五法パターン検出結果 ===")
for key, value in summary.items():
    if 'percentage' in key:
        print(f"{key}: {value:.2f}%")
    else:
        print(f"{key}: {value}")

# 赤三兵パターンの検出
aka_sanpei_patterns = sakata.detect_aka_sanpei()
print(f"\n赤三兵パターン検出: {len(aka_sanpei_patterns)}件")

# 黒三兵パターンの検出
kuro_sanpei_patterns = sakata.detect_kuro_sanpei()
print(f"黒三兵パターン検出: {len(kuro_sanpei_patterns)}件")


In [ ]:
# 赤三兵パターンの可視化（最初の検出例）
if len(aka_sanpei_patterns) > 0:
    first_aka_sanpei_idx = aka_sanpei_patterns.index[0]
    start_idx = max(0, df.index.get_loc(first_aka_sanpei_idx) - 20)
    end_idx = min(len(df), df.index.get_loc(first_aka_sanpei_idx) + 20)
    
    print(f"赤三兵パターン例: {first_aka_sanpei_idx}")
    
    # チャート表示
    chart_data = df.iloc[start_idx:end_idx]
    apd = mpf.make_addplot(chart_data[['bb_up', 'bb_mid', 'bb_low']])
    
    mpf.plot(
        chart_data[['Open', 'High', 'Low', 'Close', 'Volume']], 
        type='candle', 
        addplot=apd, 
        volume=True,
        style='yahoo',
        title="赤三兵パターン例",
        figsize=(15, 8)
    )
else:
    print("赤三兵パターンが検出されませんでした")


In [ ]:
# 黒三兵パターンの可視化（最初の検出例）
if len(kuro_sanpei_patterns) > 0:
    first_kuro_sanpei_idx = kuro_sanpei_patterns.index[0]
    start_idx = max(0, df.index.get_loc(first_kuro_sanpei_idx) - 20)
    end_idx = min(len(df), df.index.get_loc(first_kuro_sanpei_idx) + 20)
    
    print(f"黒三兵パターン例: {first_kuro_sanpei_idx}")
    
    # チャート表示
    chart_data = df.iloc[start_idx:end_idx]
    apd = mpf.make_addplot(chart_data[['bb_up', 'bb_mid', 'bb_low']])
    
    mpf.plot(
        chart_data[['Open', 'High', 'Low', 'Close', 'Volume']], 
        type='candle', 
        addplot=apd, 
        volume=True,
        style='yahoo',
        title="黒三兵パターン例",
        figsize=(15, 8)
    )
else:
    print("黒三兵パターンが検出されませんでした")


In [ ]:
# パターンの有効性分析
def analyze_pattern_effectiveness(df, pattern_column, days_ahead=5):
    """
    パターンの有効性を分析
    
    Args:
        df: データフレーム
        pattern_column: パターン列名
        days_ahead: 何日後の結果を見るか
        
    Returns:
        dict: 分析結果
    """
    pattern_dates = df[df[pattern_column] == 1].index
    results = []
    
    for date in pattern_dates:
        try:
            current_idx = df.index.get_loc(date)
            if current_idx + days_ahead < len(df):
                current_price = df.iloc[current_idx]['Close']
                future_price = df.iloc[current_idx + days_ahead]['Close']
                return_pct = (future_price - current_price) / current_price * 100
                results.append(return_pct)
        except:
            continue
    
    if results:
        return {
            'count': len(results),
            'avg_return': np.mean(results),
            'success_rate': len([r for r in results if r > 0]) / len(results) * 100,
            'max_return': max(results),
            'min_return': min(results)
        }
    else:
        return None

# 赤三兵パターンの有効性分析
aka_effectiveness = analyze_pattern_effectiveness(sakata.df, 'aka_sanpei', days_ahead=5)
if aka_effectiveness:
    print("=== 赤三兵パターンの有効性 (5日後) ===")
    for key, value in aka_effectiveness.items():
        if 'rate' in key or 'return' in key:
            print(f"{key}: {value:.2f}%")
        else:
            print(f"{key}: {value}")

# 黒三兵パターンの有効性分析
kuro_effectiveness = analyze_pattern_effectiveness(sakata.df, 'kuro_sanpei', days_ahead=5)
if kuro_effectiveness:
    print("\n=== 黒三兵パターンの有効性 (5日後) ===")
    for key, value in kuro_effectiveness.items():
        if 'rate' in key or 'return' in key:
            print(f"{key}: {value:.2f}%")
        else:
            print(f"{key}: {value}")


In [ ]:
# パターン発生頻度の時系列分析
df_analysis = sakata.df.copy()

# 月次の集計
df_analysis['month'] = df_analysis.index.to_period('M')
monthly_patterns = df_analysis.groupby('month').agg({
    'aka_sanpei': 'sum',
    'kuro_sanpei': 'sum'
}).reset_index()

# 可視化
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.plot(monthly_patterns['month'].astype(str), monthly_patterns['aka_sanpei'], 
         marker='o', label='赤三兵', color='red')
plt.plot(monthly_patterns['month'].astype(str), monthly_patterns['kuro_sanpei'], 
         marker='s', label='黒三兵', color='blue')
plt.title('酒田五法パターンの月次発生頻度')
plt.xlabel('月')
plt.ylabel('発生回数')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# パターン発生時の価格とボリンジャーバンドの関係
plt.subplot(1, 2, 2)
if len(aka_sanpei_patterns) > 0:
    plt.scatter(aka_sanpei_patterns['Close'], 
               (aka_sanpei_patterns['Close'] - aka_sanpei_patterns['bb_mid']) / aka_sanpei_patterns['bb_mid'] * 100,
               alpha=0.6, color='red', label='赤三兵', s=50)

if len(kuro_sanpei_patterns) > 0:
    plt.scatter(kuro_sanpei_patterns['Close'], 
               (kuro_sanpei_patterns['Close'] - kuro_sanpei_patterns['bb_mid']) / kuro_sanpei_patterns['bb_mid'] * 100,
               alpha=0.6, color='blue', label='黒三兵', s=50)

plt.title('パターン発生時の価格とボリンジャーバンド中心線からの乖離')
plt.xlabel('価格')
plt.ylabel('BB中心線からの乖離率 (%)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 結果の保存
analysis_results = {
    'total_data_points': len(df),
    'aka_sanpei_patterns': len(aka_sanpei_patterns),
    'kuro_sanpei_patterns': len(kuro_sanpei_patterns),
    'aka_sanpei_effectiveness': aka_effectiveness,
    'kuro_sanpei_effectiveness': kuro_effectiveness
}

results_path = "../../results/sakata_pattern_analysis.csv"
os.makedirs(os.path.dirname(results_path), exist_ok=True)

# 結果をフラット化してCSV保存用に準備
flat_results = {
    'total_data_points': analysis_results['total_data_points'],
    'aka_sanpei_patterns': analysis_results['aka_sanpei_patterns'],
    'kuro_sanpei_patterns': analysis_results['kuro_sanpei_patterns'],
}

if aka_effectiveness:
    for key, value in aka_effectiveness.items():
        flat_results[f'aka_sanpei_{key}'] = value

if kuro_effectiveness:
    for key, value in kuro_effectiveness.items():
        flat_results[f'kuro_sanpei_{key}'] = value

pd.DataFrame([flat_results]).to_csv(results_path, index=False)
print(f"分析結果を保存しました: {results_path}")
